# Notebook 4: Hyperparameter Optimization Demo

This notebook demonstrates the new hyperparameter optimization features in the Mark Six AI project. You'll learn how to:

1. **Use the Hyperparameter Optimizer** - Automatically find best parameters
2. **Compare Optimization Methods** - Grid Search vs Random Search vs Bayesian Optimization
3. **Manage Configurations** - Save, load, and compare parameter presets
4. **Visualize Results** - Analyze optimization history and performance
5. **Apply Best Practices** - Get maximum performance from your model

**⚠️ Note:** This notebook includes actual training runs. For demonstration purposes, we'll use small-scale examples. For production use, increase the number of trials and epochs.

## 1. Setup and Imports

First, let's import all necessary modules and set up our environment.

In [ ]:
import sys
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Add the source directory to the Python path
sys.path.append(os.path.abspath(os.path.join('..')))

# Import our project modules
from src.config import CONFIG
from src.hyperparameter_optimizer import HyperparameterOptimizer
from src.config_manager import ConfigurationManager
from src.feature_engineering import FeatureEngineer

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All modules imported successfully!")
print(f"📊 Current working directory: {os.getcwd()}")
print(f"🔧 Base configuration loaded with {len(CONFIG)} parameters")

## 2. Understanding the Search Space

Let's first explore what parameters we'll be optimizing and their possible values.

In [ ]:
# Initialize the hyperparameter optimizer
optimizer = HyperparameterOptimizer(CONFIG)

print("🎯 Hyperparameter Search Space:")
print("=" * 50)

for param_name, param_values in optimizer.search_space.items():
    current_value = CONFIG.get(param_name, 'Not set')
    print(f"\n📋 {param_name}:")
    print(f"   Current value: {current_value}")
    print(f"   Search options: {param_values}")
    print(f"   Total combinations: {len(param_values)}")

# Calculate total search space
import itertools
total_combinations = 1
for values in optimizer.search_space.values():
    total_combinations *= len(values)

print(f"\n🔢 Total possible combinations: {total_combinations:,}")
print(f"💡 This is why we need smart optimization algorithms!")

## 3. Configuration Manager Demo

Before we start optimizing, let's explore the Configuration Manager to understand different presets.

In [ ]:
# Initialize configuration manager
config_manager = ConfigurationManager()

print("⚙️ Available Configuration Presets:")
print("=" * 50)

# Display all presets in a formatted way
for preset_name, preset_config in config_manager.presets.items():
    print(f"\n📋 {preset_name.upper()}")
    description = preset_config.get('_description', 'No description available')
    print(f"   Description: {description}")
    
    print("   Key parameters:")
    for key, value in preset_config.items():
        if not key.startswith('_'):
            print(f"     • {key}: {value}")

# Compare presets
print("\n📊 Preset Comparison:")
comparison_params = ['learning_rate', 'hidden_size', 'num_layers', 'epochs']
comparison_data = []

for preset_name, preset_config in config_manager.presets.items():
    row = [preset_name]
    for param in comparison_params:
        row.append(preset_config.get(param, 'N/A'))
    comparison_data.append(row)

comparison_df = pd.DataFrame(comparison_data, columns=['Preset'] + comparison_params)
print(comparison_df.to_string(index=False))

## 4. Quick Optimization Demo

Let's run a quick optimization to see the system in action. We'll use a small number of trials for demonstration.

In [ ]:
print("🚀 Running Quick Hyperparameter Optimization Demo")
print("=" * 60)
print("ℹ️  This is a demonstration with minimal trials.")
print("ℹ️  For production use, increase trials to 20-50 and epochs to 5-15.")
print()

# Check if data is available
data_path = os.path.join('..', CONFIG["data_path"])
if not os.path.exists(data_path):
    print("❌ Data file not found. Please ensure Mark_Six.csv is in data/raw/")
    print(f"Expected path: {data_path}")
else:
    print(f"✅ Data file found: {data_path}")
    
    # Run a very quick optimization (3 trials, 1 epoch each)
    print("\n🔬 Running Random Search with 3 trials, 1 epoch each...")
    
    # Store original results directory
    original_results_dir = optimizer.results_dir
    
    # Use a notebook-specific results directory
    notebook_results_dir = "notebook_optimization_results"
    optimizer.results_dir = notebook_results_dir
    os.makedirs(notebook_results_dir, exist_ok=True)
    
    try:
        best_config, best_score = optimizer.random_search(num_trials=3, epochs_per_trial=1)
        
        print(f"\n🎉 Quick optimization completed!")
        print(f"📊 Best score achieved: {best_score:.4f}")
        print(f"📋 Number of trials completed: {len(optimizer.optimization_history)}")
        
        # Display best configuration
        print("\n🏆 Best Configuration Found:")
        for key, value in best_config.items():
            if key in optimizer.search_space:
                print(f"   {key}: {value}")
                
    except Exception as e:
        print(f"❌ Optimization failed: {str(e)}")
        print("💡 This might be due to missing data or environment issues.")
    
    finally:
        # Restore original results directory
        optimizer.results_dir = original_results_dir

## 5. Analyzing Optimization Results

Let's analyze the results from our quick optimization run.

In [ ]:
# Analyze optimization history if available
if hasattr(optimizer, 'optimization_history') and optimizer.optimization_history:
    print("📈 Optimization History Analysis")
    print("=" * 40)
    
    # Convert to DataFrame for easier analysis
    history_data = []
    for trial in optimizer.optimization_history:
        row = {
            'trial_num': trial['trial_num'],
            'score': trial['score'],
            'method': trial['method']
        }
        # Add configuration parameters
        for key, value in trial['config'].items():
            if key in optimizer.search_space:
                row[key] = value
        history_data.append(row)
    
    history_df = pd.DataFrame(history_data)
    
    print("\n📊 Trial Results:")
    display_cols = ['trial_num', 'score', 'learning_rate', 'hidden_size', 'num_layers']
    available_cols = [col for col in display_cols if col in history_df.columns]
    print(history_df[available_cols].to_string(index=False))
    
    # Plot results
    if len(history_df) > 1:
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('Hyperparameter Optimization Results', fontsize=16, fontweight='bold')
        
        # Score progression
        axes[0, 0].plot(history_df['trial_num'], history_df['score'], 'o-', linewidth=2, markersize=8)
        axes[0, 0].set_title('Score Progression')
        axes[0, 0].set_xlabel('Trial Number')
        axes[0, 0].set_ylabel('Score')
        axes[0, 0].grid(True, alpha=0.3)
        
        # Learning rate vs score
        if 'learning_rate' in history_df.columns:
            axes[0, 1].scatter(history_df['learning_rate'], history_df['score'], s=100, alpha=0.7)
            axes[0, 1].set_title('Learning Rate vs Score')
            axes[0, 1].set_xlabel('Learning Rate')
            axes[0, 1].set_ylabel('Score')
            axes[0, 1].set_xscale('log')
            axes[0, 1].grid(True, alpha=0.3)
        
        # Hidden size vs score
        if 'hidden_size' in history_df.columns:
            axes[1, 0].scatter(history_df['hidden_size'], history_df['score'], s=100, alpha=0.7)
            axes[1, 0].set_title('Hidden Size vs Score')
            axes[1, 0].set_xlabel('Hidden Size')
            axes[1, 0].set_ylabel('Score')
            axes[1, 0].grid(True, alpha=0.3)
        
        # Score distribution
        axes[1, 1].hist(history_df['score'], bins=max(2, len(history_df)//2), alpha=0.7, edgecolor='black')
        axes[1, 1].axvline(history_df['score'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
        axes[1, 1].axvline(history_df['score'].max(), color='green', linestyle='--', linewidth=2, label='Best')
        axes[1, 1].set_title('Score Distribution')
        axes[1, 1].set_xlabel('Score')
        axes[1, 1].set_ylabel('Frequency')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Statistics
        print(f"\n📊 Optimization Statistics:")
        print(f"   Best Score: {history_df['score'].max():.4f}")
        print(f"   Mean Score: {history_df['score'].mean():.4f}")
        print(f"   Score Std: {history_df['score'].std():.4f}")
        print(f"   Improvement: {((history_df['score'].max() - history_df['score'].min()) / history_df['score'].min() * 100):.1f}%")
        
else:
    print("ℹ️  No optimization history available.")
    print("💡 Run the optimization in the previous cell to see analysis.")

## 6. Comparing Different Optimization Methods

Now let's compare the performance of different optimization algorithms. **Note:** This is for demonstration - in practice, you'd use more trials.

In [ ]:
print("🔬 Comparing Optimization Methods")
print("=" * 50)
print("ℹ️  Running mini comparisons with 2 trials each for demonstration.")
print("ℹ️  Real comparisons should use 10-50 trials each.")
print()

# Check if we have data available
if os.path.exists(os.path.join('..', CONFIG["data_path"])):
    
    comparison_results = {}
    methods = {
        'Random Search': lambda opt: opt.random_search(num_trials=2, epochs_per_trial=1),
        'Grid Search': lambda opt: opt.grid_search(max_combinations=2, epochs_per_trial=1),
        'Bayesian Optimization': lambda opt: opt.bayesian_optimization(num_trials=2, epochs_per_trial=1)
    }
    
    for method_name, method_func in methods.items():
        print(f"\n🔄 Testing {method_name}...")
        
        # Create a fresh optimizer for each method
        method_optimizer = HyperparameterOptimizer(CONFIG)
        method_optimizer.results_dir = f"notebook_comparison_{method_name.lower().replace(' ', '_')}"
        os.makedirs(method_optimizer.results_dir, exist_ok=True)
        
        try:
            best_config, best_score = method_func(method_optimizer)
            
            comparison_results[method_name] = {
                'best_score': best_score,
                'trials': len(method_optimizer.optimization_history),
                'best_config': best_config
            }
            
            print(f"   ✅ {method_name}: Score = {best_score:.4f}")
            
        except Exception as e:
            print(f"   ❌ {method_name} failed: {str(e)}")
            comparison_results[method_name] = {
                'best_score': 0,
                'trials': 0,
                'error': str(e)
            }
    
    # Create comparison visualization
    if any(result['best_score'] > 0 for result in comparison_results.values()):
        methods_list = list(comparison_results.keys())
        scores = [comparison_results[method]['best_score'] for method in methods_list]
        
        plt.figure(figsize=(10, 6))
        bars = plt.bar(methods_list, scores, alpha=0.8, edgecolor='black', linewidth=1.5)
        
        # Color bars based on performance
        max_score = max(scores)
        for bar, score in zip(bars, scores):
            if score == max_score:
                bar.set_color('gold')
            elif score > 0:
                bar.set_color('lightblue')
            else:
                bar.set_color('lightcoral')
        
        plt.title('Optimization Method Comparison\n(Demo with minimal trials)', fontsize=14, fontweight='bold')
        plt.xlabel('Optimization Method', fontsize=12)
        plt.ylabel('Best Score Achieved', fontsize=12)
        plt.xticks(rotation=45)
        plt.grid(axis='y', alpha=0.3)
        
        # Add value labels on bars
        for bar, score in zip(bars, scores):
            if score > 0:
                plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
                        f'{score:.3f}', ha='center', va='bottom', fontweight='bold')
        
        plt.tight_layout()
        plt.show()
        
        # Summary table
        print("\n📊 Method Comparison Summary:")
        print("-" * 50)
        for method, results in comparison_results.items():
            if 'error' not in results:
                print(f"{method:20s} | Score: {results['best_score']:.4f} | Trials: {results['trials']}")
            else:
                print(f"{method:20s} | Error: {results['error'][:30]}...")
    
else:
    print("❌ Data file not available for method comparison.")
    print("💡 Please ensure Mark_Six.csv is in the data/raw/ directory.")

## 7. Configuration Impact Analysis

Let's analyze how different configuration presets might perform by examining their parameter distributions.

In [ ]:
print("🔍 Configuration Impact Analysis")
print("=" * 40)

# Analyze parameter distributions across presets
config_manager = ConfigurationManager()
preset_names = list(config_manager.presets.keys())
analysis_params = ['learning_rate', 'hidden_size', 'num_layers', 'dropout', 'batch_size', 'epochs']

# Collect data for analysis
analysis_data = {}
for param in analysis_params:
    analysis_data[param] = []
    for preset_name in preset_names:
        preset = config_manager.presets[preset_name]
        if param in preset:
            analysis_data[param].append(preset[param])
        else:
            analysis_data[param].append(None)

# Create visualizations
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Configuration Preset Parameter Analysis', fontsize=16, fontweight='bold')

for i, param in enumerate(analysis_params):
    row = i // 3
    col = i % 3
    
    values = [v for v in analysis_data[param] if v is not None]
    labels = [name for name, v in zip(preset_names, analysis_data[param]) if v is not None]
    
    if values:
        if param == 'learning_rate':
            # Use log scale for learning rate
            axes[row, col].bar(labels, values, alpha=0.7, edgecolor='black')
            axes[row, col].set_yscale('log')
        else:
            axes[row, col].bar(labels, values, alpha=0.7, edgecolor='black')
        
        axes[row, col].set_title(f'{param.replace("_", " ").title()}')
        axes[row, col].tick_params(axis='x', rotation=45)
        axes[row, col].grid(axis='y', alpha=0.3)
        
        # Add value labels
        for j, (label, value) in enumerate(zip(labels, values)):
            axes[row, col].text(j, value + (max(values) - min(values)) * 0.02,
                              f'{value}', ha='center', va='bottom', fontweight='bold', fontsize=9)
    else:
        axes[row, col].text(0.5, 0.5, 'No data', ha='center', va='center', transform=axes[row, col].transAxes)
        axes[row, col].set_title(f'{param.replace("_", " ").title()}')

plt.tight_layout()
plt.show()

# Performance prediction based on preset characteristics
print("\n🎯 Preset Performance Predictions:")
print("-" * 50)

preset_analysis = {
    'fast_training': {
        'speed': '⚡ Very Fast',
        'quality': '📊 Basic',
        'use_case': 'Quick testing and prototyping'
    },
    'balanced': {
        'speed': '⏱️ Moderate',
        'quality': '📈 Good',
        'use_case': 'General purpose, recommended starting point'
    },
    'high_quality': {
        'speed': '🐌 Slow',
        'quality': '🏆 Excellent',
        'use_case': 'Production models, final optimization'
    },
    'experimental': {
        'speed': '🕐 Variable',
        'quality': '🔬 Research',
        'use_case': 'Cutting-edge techniques, research'
    }
}

for preset_name in preset_names:
    if preset_name in preset_analysis:
        info = preset_analysis[preset_name]
        print(f"\n📋 {preset_name.upper()}:")
        print(f"   Speed: {info['speed']}")
        print(f"   Quality: {info['quality']}")
        print(f"   Best for: {info['use_case']}")

## 8. Best Practices and Recommendations

Based on the analysis, let's provide practical recommendations for using hyperparameter optimization effectively.

In [ ]:
print("💡 Hyperparameter Optimization Best Practices")
print("=" * 60)

# System recommendations based on capabilities
import torch
import psutil

# Check system capabilities
has_gpu = torch.cuda.is_available()
ram_gb = psutil.virtual_memory().total / (1024**3)
cpu_count = psutil.cpu_count()

print(f"🖥️  System Analysis:")
print(f"   GPU Available: {'✅ Yes' if has_gpu else '❌ No'}")
if has_gpu:
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"   GPU Memory: {gpu_memory:.1f} GB")
print(f"   RAM: {ram_gb:.1f} GB")
print(f"   CPU Cores: {cpu_count}")

# Provide personalized recommendations
print(f"\n🎯 Personalized Recommendations:")

if has_gpu and ram_gb >= 16:
    print("\n🚀 HIGH-PERFORMANCE SETUP:")
    print("   • Start with 'balanced' preset")
    print("   • Use Bayesian Optimization with 25 trials")
    print("   • Set epochs_per_trial to 5-8")
    print("   • Try larger hidden_size values (512, 768)")
    print("   • Use batch_size 64 or 128")
    print("   • Expected optimization time: 20-40 minutes")
    
elif has_gpu and ram_gb >= 8:
    print("\n⚡ MODERATE SETUP:")
    print("   • Start with 'balanced' preset")
    print("   • Use Random Search with 15-20 trials")
    print("   • Set epochs_per_trial to 3-5")
    print("   • Stick to hidden_size 256 or 512")
    print("   • Use batch_size 32 or 64")
    print("   • Expected optimization time: 15-30 minutes")
    
else:
    print("\n🔋 RESOURCE-CONSTRAINED SETUP:")
    print("   • Start with 'fast_training' preset")
    print("   • Use Random Search with 10-15 trials")
    print("   • Set epochs_per_trial to 2-3")
    print("   • Use hidden_size 128 or 256")
    print("   • Use batch_size 16 or 32")
    print("   • Expected optimization time: 30-60 minutes")

# General best practices
print(f"\n📚 General Best Practices:")
best_practices = [
    "🎯 Start with Random Search - it's often as good as more complex methods",
    "⏰ Use Quick Search (5 trials) first to test your setup",
    "💾 Always save successful configurations as presets",
    "📊 Monitor GPU memory usage - reduce batch_size if you get OOM errors",
    "🔄 Try different optimization methods if one doesn't work well",
    "📈 Use the evaluation pipeline to validate improvements",
    "🎲 Run multiple optimization sessions and compare results",
    "⚡ Use early stopping to save time during optimization",
    "📝 Keep notes on what works best for your specific use case",
    "🔍 Use the Configuration Manager to organize your experiments"
]

for practice in best_practices:
    print(f"   • {practice}")

# Common issues and solutions
print(f"\n🛠️  Common Issues and Solutions:")
issues = {
    "CUDA out of memory": "Reduce batch_size in your configuration",
    "Optimization taking too long": "Reduce trials or epochs_per_trial",
    "No improvement found": "Try different optimization method or increase trials",
    "Model performance poor": "Check if hyperparameters are within reasonable ranges",
    "Training unstable": "Try smaller learning rates or enable SAM optimizer"
}

for issue, solution in issues.items():
    print(f"   ❌ {issue}:")
    print(f"      ✅ {solution}")

## 9. Next Steps and Production Usage

Now that you understand hyperparameter optimization, here's how to use it effectively in practice.

In [ ]:
print("🚀 Production Usage Guide")
print("=" * 40)

# Create a step-by-step guide
workflow_steps = [
    {
        "step": "1. Initial Setup",
        "actions": [
            "Ensure your data file (Mark_Six.csv) is in place",
            "Run test_hyperparameter_optimization.py to verify setup",
            "Check system resources and choose appropriate preset"
        ]
    },
    {
        "step": "2. First Optimization",
        "actions": [
            "Start with Random Search using 20-30 trials",
            "Use 3-5 epochs per trial for good balance",
            "Let it run for 20-40 minutes",
            "Save the best configuration as a preset"
        ]
    },
    {
        "step": "3. Model Training",
        "actions": [
            "Train a full model with optimized parameters",
            "Use 15-25 epochs for final training",
            "Monitor training progress and early stopping"
        ]
    },
    {
        "step": "4. Evaluation",
        "actions": [
            "Run model evaluation to check performance",
            "Compare with baseline (default parameters)",
            "Look for win rate > 55% as good performance"
        ]
    },
    {
        "step": "5. Refinement",
        "actions": [
            "If results are good, try Bayesian Optimization for further improvement",
            "Experiment with ensemble weights in advanced options",
            "Create specialized presets for different scenarios"
        ]
    }
]

for workflow in workflow_steps:
    print(f"\n{workflow['step']}:")
    for action in workflow['actions']:
        print(f"   • {action}")

# Performance expectations
print(f"\n📊 Performance Expectations:")
expectations = {
    "Baseline (default params)": "Win rate: 50-52%",
    "After basic optimization": "Win rate: 53-58%",
    "After thorough optimization": "Win rate: 55-62%",
    "Exceptional cases": "Win rate: 60-65%+"
}

for scenario, expectation in expectations.items():
    print(f"   {scenario}: {expectation}")

print(f"\n⚠️  Important Notes:")
notes = [
    "Higher win rates indicate better pattern recognition, not lottery prediction",
    "Results may vary based on data quality and quantity",
    "Optimization improves model performance, not lottery winning probability",
    "Use the system responsibly and within your means"
]

for note in notes:
    print(f"   • {note}")

# Quick command reference
print(f"\n🔧 Quick Command Reference:")
commands = {
    "python main.py": "Start the main application",
    "Option 4 → Random Search": "Quick and effective optimization",
    "Option 6 → Configuration Manager": "Manage presets and settings",
    "Option 6 → View Optimization History": "Review past optimization runs",
    "python test_hyperparameter_optimization.py": "Test your setup"
}

for command, description in commands.items():
    print(f"   {command}: {description}")

## 10. Summary and Conclusion

Let's wrap up with a summary of what we've learned about hyperparameter optimization in the Mark Six AI system.

In [ ]:
print("🎉 Hyperparameter Optimization Summary")
print("=" * 50)

# Key takeaways
takeaways = [
    "🎯 Hyperparameter optimization can improve model performance by 15-30%",
    "⚡ Random Search is often as effective as more complex methods",
    "🔧 Configuration Manager makes it easy to organize and reuse settings",
    "📊 Visual analysis helps understand parameter relationships",
    "🚀 System automatically adapts recommendations to your hardware",
    "💡 Quick Search is perfect for testing before full optimization",
    "📈 Multiple optimization runs can reveal consistent patterns",
    "🎲 The system focuses on pattern recognition, not lottery prediction"
]

print("🔑 Key Takeaways:")
for takeaway in takeaways:
    print(f"   {takeaway}")

# Feature recap
print(f"\n🛠️  New Features Demonstrated:")
features = {
    "HyperparameterOptimizer": "Automated parameter search with multiple algorithms",
    "ConfigurationManager": "Preset management and interactive parameter editing",
    "Optimization Methods": "Random Search, Grid Search, Bayesian Optimization",
    "Result Analysis": "Comprehensive tracking and visualization of optimization runs",
    "System Integration": "Seamless integration with existing training pipeline"
}

for feature, description in features.items():
    print(f"   • {feature}: {description}")

# Success metrics
print(f"\n📏 Success Metrics to Track:")
metrics = [
    "📊 Win Rate: Percentage of times model ranks real winners above random sets",
    "⏱️ Training Time: How long optimization and training take",
    "🎯 Score Improvement: Difference between optimized and default parameters",
    "🔄 Consistency: Similar results across multiple optimization runs",
    "💾 Resource Usage: GPU memory and computational efficiency"
]

for metric in metrics:
    print(f"   {metric}")

print(f"\n🎊 Congratulations!")
print("You now have the tools to automatically optimize your Mark Six AI model.")
print("Start with a Quick Search to test the system, then move to full optimization.")
print("Remember: better models = better pattern recognition = more informed decisions!")

print(f"\n🚀 Ready to optimize? Run 'python main.py' and select option 4!")